In [1]:
import genjax
from genjax import dippl
from genjax import gensp
import jax
import jax.numpy as jnp
import adevjax

console = genjax.pretty(show_locals=True)
key = jax.random.PRNGKey(314159)


@genjax.gen
def flipper(p):
    flip = dippl.flip_enum(p) @ "flip"


# Lift to GenSP.
lifted_model = gensp.choice_map_distribution(flipper, genjax.select("flip"), None)


def debug_loss(key, p):
    @dippl.loss
    def flip_variational_loss(p):
        v = dippl.upper(lifted_model)(p)

    loss = flip_variational_loss.jvp_estimate(key, (p,), (1.0,))
    return loss


jaxpr = jax.make_jaxpr(debug_loss)(key, 0.3)
print(jaxpr)

I0000 00:00:1696194932.106387   39454 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


{ lambda ; a:u32[2] b:f32[]. let
    c:key<fry>[] = random_seed[impl=fry] 0
    d:u32[2] = random_unwrap c
    e:u32[2] = sample[
      in_tree=PyTreeDef((CustomNode(GrabKey[()], []), *))
      jaxpr={ lambda ; f:u32[2]. let  in (f,) }
      kont=<function CPSInterpreter._eval_jaxpr_cps.<locals>.eval_jaxpr_recurse.<locals>.kont at 0x7f44a6b20ea0>
      num_consts=0
      out_tree=PyTreeDef(*)
    ] d
  in (e,) }
